In [ ]:
import glob as gb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
image_height = 351
image_weight = 351
batch_size = 10

In [ ]:
path = "/content/drive/MyDrive/Vehicle recognition/Dataset"

In [ ]:
training_data = keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size = 16,
    image_size =(351,351),

    shuffle = True,
    seed =123,
    subset ='training',
    validation_split=0.15
    )
validation_data =keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size = 16,
    image_size =(351,351),

    shuffle = True,
    seed =123,
    validation_split =0.15,
    subset ='validation'

    )

Found 400 files belonging to 4 classes.
Using 340 files for training.
Found 400 files belonging to 4 classes.
Using 60 files for validation.


In [ ]:
resnet_model = Sequential()

pretrained_model= keras.applications.ResNet50(include_top=False,
                   input_shape=(351,351,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(4, activation='softmax'))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
resnet_model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.00009), metrics=['accuracy'])

In [ ]:
epochs=10
history = resnet_model.fit(

  training_data,
  validation_data=validation_data,
  epochs=epochs
)

Epoch 1/10
22/22 [==============================] - 229s 9s/step - loss: 1.5187 - accuracy: 0.2941 - val_loss: 1.3185 - val_accuracy: 0.3333
Epoch 2/10
22/22 [==============================] - 197s 9s/step - loss: 1.4210 - accuracy: 0.3500 - val_loss: 1.2481 - val_accuracy: 0.3833
Epoch 3/10
22/22 [==============================] - 197s 9s/step - loss: 1.3460 - accuracy: 0.4088 - val_loss: 1.1912 - val_accuracy: 0.4667
Epoch 4/10
22/22 [==============================] - 201s 9s/step - loss: 1.2836 - accuracy: 0.4559 - val_loss: 1.1440 - val_accuracy: 0.5167
Epoch 5/10
22/22 [==============================] - 196s 9s/step - loss: 1.2295 - accuracy: 0.4971 - val_loss: 1.1076 - val_accuracy: 0.5333
Epoch 6/10
22/22 [==============================] - 199s 9s/step - loss: 1.1847 - accuracy: 0.5235 - val_loss: 1.0733 - val_accuracy: 0.6000
Epoch 7/10
22/22 [==============================] - 207s 9s/step - loss: 1.1465 - accuracy: 0.5382 - val_loss: 1.0428 - val_accuracy: 0.6333
Epoch 8/10
22

In [ ]:
# Save the trained model to an .h5 file
resnet_model.save('vehicle_reco_model.h5')

print("Trained model saved as 'trained_vehicle_model.h5'")

Trained model saved as 'trained_vehicle_model.h5'


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [ ]:
!pip install opencv-python-headless

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from google.colab.patches import cv2_imshow

# Load the trained ResNet-based model
model_path = '/content/vehicle_reco_model.h5'  # Replace with the actual path
model = load_model(model_path)

# Labels for vehicle classes
class_names = ['car', 'motorcycle', 'truck', 'bus']

# Open video capture
video_path = '/content/cars.mp4'  # Replace with the actual path
video_capture = cv2.VideoCapture(video_path)

# Read video dimensions
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))

# Set up video writer for the output video
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Use appropriate codec (e.g., XVID, MP4V)
frame_size = (frame_width, frame_height)
video_writer = cv2.VideoWriter(output_path, fourcc, 30, frame_size)  # Adjust frame rate as needed

while True:
    # Read a frame from the video
    ret, frame = video_capture.read()

    if not ret:
        print("End of video")
        break

    # Preprocess the frame for input to the model
    resized_frame = cv2.resize(frame, (image_height, image_weight))
    preprocessed_frame = preprocess_input(np.expand_dims(resized_frame, axis=0))

    # Use the model to predict the class of the vehicle
    predictions = model.predict(preprocessed_frame)
    predicted_class = np.argmax(predictions)

    # Get the class name from the list of class names
    vehicle_class = class_names[predicted_class]

    # Display the recognized class on the frame
    cv2.putText(frame, f'Class: {vehicle_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame using cv2_imshow()
    cv2_imshow(frame)

    # Write the frame to the output video
    video_writer.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video writer and capture
video_writer.release()
video_capture.release()
cv2.destroyAllWindows()
